In [2]:
# Data Analysis
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt
import pickle as pkl
from scipy import sparse

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud, STOPWORDS

# Text Processing
import re
import itertools
import string
import collections
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Machine Learning packages
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import sklearn.cluster as cluster
from sklearn.manifold import TSNE

# Model training and evaluation
from sklearn.model_selection import train_test_split

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance

#Metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.metrics import classification_report

# Ignore noise warning
import warnings
warnings.filterwarnings("ignore")

In [3]:
df_wine = pd.read_csv('./wine400_list.csv')

In [5]:
df_wine = df_wine.rename(columns={'Unnamed: 0':'주종'}) # 컬럼명 변경 

In [6]:
df_wine.head()

,주종,중분류,상품명,생산국가,상세정보
0,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,생산자\nGCF 그룹 GCF Group\n국가/생산지역\n프랑스(France) > ...
1,wine,화이트,"가트, 리슬링",호주,생산자\n가트 와인즈 Gatt Wines\n국가/생산지역\n호주(Australia)...
2,wine,레드,갈로 리빙스톤 콩코드,미국,생산자\n갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Ga...
3,wine,주정강화,글로리아 토니 포트 와인,포르투갈,생산자\n빈센테 파리아 비뉴스 Vincente Faria VInhos\n국가/생산지...
4,wine,레드,"기욤 르탕, 루비 루즈",프랑스,생산자\n기욤 르탕 Guillaume Letang\n국가/생산지역\n프랑스(Fran...


In [ ]:
# 상세정보 전처리 -> 기준으로 삼을 수 있는게 공백밖에 없음 (글자수가 각각 다르기때문)
# 우리가 필요한 정보는 생산자, 주요품종, 음용온도, 알코올, 추천음식 

In [7]:
detail_split = df_wine["상세정보"].str.split("\n")

In [8]:
# wine데이터프레임에 필요한 내용들 추가하기 
df_wine["생산자"] = detail_split.str.get(1)
df_wine["주요품종"] = detail_split.str.get(5)

In [9]:
# 잘 들어온 것을 확인할 수 있음 
df_wine

,주종,중분류,상품명,생산국가,상세정보,생산자,주요품종
0,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,생산자\nGCF 그룹 GCF Group\n국가/생산지역\n프랑스(France) > ...,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)"
1,wine,화이트,"가트, 리슬링",호주,생산자\n가트 와인즈 Gatt Wines\n국가/생산지역\n호주(Australia)...,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%
2,wine,레드,갈로 리빙스톤 콩코드,미국,생산자\n갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Ga...,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord)
3,wine,주정강화,글로리아 토니 포트 와인,포르투갈,생산자\n빈센테 파리아 비뉴스 Vincente Faria VInhos\n국가/생산지...,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca..."
4,wine,레드,"기욤 르탕, 루비 루즈",프랑스,생산자\n기욤 르탕 Guillaume Letang\n국가/생산지역\n프랑스(Fran...,기욤 르탕 Guillaume Letang,그르나슈 (Grenache)
...,...,...,...,...,...,...,...
394,wine,레드,"매티스, 그르나슈",미국,생산자\n매티스 와이너리 Mathis Winery\n국가/생산지역\n미국(U.S.A...,매티스 와이너리 Mathis Winery,"그르나슈 (Grenache) 82%, 까리냥 (Carignan/Carignane) ..."
395,wine,화이트,"메종 카스텔, 소비뇽 블랑",프랑스,생산자\n메종 카스텔 Maison Castel\n국가/생산지역\n프랑스(France...,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc)
396,wine,스파클링,"메종 카스텔, 퀴베 블랑쉬 브뤼",프랑스,생산자\n메종 카스텔 Maison Castel\n국가/생산지역\n프랑스(France...,메종 카스텔 Maison Castel,샤르도네 (Chardonnay)
397,wine,화이트,"메종 카스텔, 푸이 퓌메",프랑스,생산자\n메종 카스텔 Maison Castel\n국가/생산지역\n프랑스(France...,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc)


In [98]:
detail_split = df_wine["상세정보"].str.split("\n", expand=True) 

In [99]:
detail_split 

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,생산자,GCF 그룹 GCF Group,국가/생산지역,프랑스(France) > 서던 프랑스(Southern France) > 랑그독 루씨...,주요품종,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",스타일,Languedoc-Roussillon Red,등급,Pays d'Oc IGP,...,None,None,None,None,None,None,None,None,None,None
1,생산자,가트 와인즈 Gatt Wines,국가/생산지역,호주(Australia) > 사우스 오스트레일리아(South Australia) >...,주요품종,리슬링 (Riesling) 100%,음용온도,10~12 ℃,추천음식,"샐러드, 해산물, 치즈 등과 잘 어울린다.",...,None,None,None,None,None,None,None,None,None,None
2,생산자,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,국가/생산지역,미국(U.S.A),주요품종,콩코드 (Concord),음용온도,14~16 ℃,추천음식,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",...,None,None,None,None,None,None,None,None,None,None
3,생산자,빈센테 파리아 비뉴스 Vincente Faria VInhos,국가/생산지역,포르투갈(Portugal) > 도우루(Douro),주요품종,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",알코올,20~21 %,음용온도,14~16 ℃,...,None,None,None,None,None,None,None,None,None,None
4,생산자,기욤 르탕 Guillaume Letang,국가/생산지역,프랑스(France) > 서던 프랑스(Southern France) > 랑그독 루씨...,주요품종,그르나슈 (Grenache),스타일,Languedoc-Roussillon Red,숙성,프랑스 전통의 오크 통에서 3개월이상 숙성.,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,생산자,매티스 와이너리 Mathis Winery,국가/생산지역,미국(U.S.A) > 캘리포니아(California) > 소노마 카운티(Sonoma...,주요품종,"그르나슈 (Grenache) 82%, 까리냥 (Carignan/Carignane) ...",등급,Sonoma Valley AVA,알코올,14~15 %,...,None,None,None,None,None,None,None,None,None,None
395,생산자,메종 카스텔 Maison Castel,국가/생산지역,프랑스(France) > 사우스웨스트 프랑스(Southwest France) > 꼬...,주요품종,소비뇽 블랑 (Sauvignon Blanc),스타일,Languedoc-Roussillon White,양조,특정 온도에서 2개월 동안 발효,...,가자주류,None,None,None,None,None,None,None,None,None
396,생산자,메종 카스텔 Maison Castel,국가/생산지역,프랑스(France) > 부르고뉴(Bourgogne),주요품종,샤르도네 (Chardonnay),알코올,11~12 %,음용온도,6~8 ℃,...,None,None,None,None,None,None,None,None,None,None
397,생산자,메종 카스텔 Maison Castel,국가/생산지역,프랑스(France) > 루아르(Loire),주요품종,소비뇽 블랑 (Sauvignon Blanc),스타일,Upper Loire White,등급,Pouilly-Fumé AOC(AOP),...,*Asia Wine Trophy: Gold,수입사,가자주류,None,None,None,None,None,None,None


In [100]:
# 필요없는 내용들 None값으로 처리 
for i in range(5,29):
    detail_split.loc[detail_split[(i)] == "숙성",(i+1)] = 'None'
    detail_split.loc[detail_split[(i)] == "스타일",(i+1)] = 'None'
    detail_split.loc[detail_split[(i)] == "등급",(i+1)] = 'None'
    detail_split.loc[detail_split[(i)] == "기타정보",(i+1)] = 'None'
    detail_split.loc[detail_split[(i)] == "수입사",(i+1)] = 'None'

In [101]:
detail_split = detail_split.iloc[:,6:20] # 추가할 컬럼 선택 후 None값 들어갔는지 확인 

In [86]:
detail_split

,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,스타일,None,등급,None,알코올,13~14 %,음용온도,16~18 ℃,추천음식,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",수입사,None,None,None
1,음용온도,10~12 ℃,추천음식,"샐러드, 해산물, 치즈 등과 잘 어울린다.",기타정보,None,-international wine challenge 2013 bronze medal,-james halliday 94 points,None,None,None,None,None,None
2,음용온도,14~16 ℃,추천음식,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",None,None,None,None,None,None,None,None,None,None
3,알코올,20~21 %,음용온도,14~16 ℃,추천음식,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",수입사,None,None,None,None,None,None,None
4,스타일,None,숙성,None,알코올,14~15 %,음용온도,16~18 ℃,추천음식,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다.",수입사,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,등급,None,알코올,14~15 %,음용온도,16~18 ℃,추천음식,"덩어리 째 오븐에 구운 육류와 햄, 스테이크, 라자냐, 치즈 등과 잘 어울린다.",기타정보,None,수입사,None,None,None
395,스타일,None,양조,특정 온도에서 2개월 동안 발효,알코올,13~14 %,음용온도,8~10 ℃,추천음식,"연어 타르타르 스테이크와 같은 해산물 요리, 샐러드와 같은 야채, 생선 그릴 요리와...",기타정보,None,*Berlin Wine Trophy: Gold,수입사
396,알코올,11~12 %,음용온도,6~8 ℃,추천음식,"디저트, 조개, 생선 요리 또는 반주와 잘 어울린다.",기타정보,None,수입사,None,None,None,None,None
397,스타일,None,등급,None,양조,다양한 효모 발효를 통해 복합성을 가중,알코올,13~14 %,음용온도,8~10 ℃,추천음식,"샤프란을 곁들인 생선 스튜 요리 등 생선, 갑각류, 조개 요리, 크림 소스를 곁들인...",기타정보,None


# 음용온도, 알코올, 추천음식만 가져오면 됨 
- 7열은 알코올, 9열 음용온도, 11열은 추천음식 

In [102]:
detail_split3 = detail_split

In [103]:
detail_split3[detail_split3[6] == '음용온도'] = detail_split3[detail_split3[6] == '음용온도'].shift(2, axis=1) # 많은 데이터를 가져온 후에는 이 코드로 작업할 것! 

In [106]:
detail_split3[detail_split3[10] == '음용온도'] = detail_split3[detail_split3[10] == '음용온도'].shift(-2, axis=1)
detail_split3[detail_split3[12] == '음용온도'] = detail_split3[detail_split3[12] == '음용온도'].shift(-4, axis=1)
detail_split3[detail_split3[14] == '음용온도'] = detail_split3[detail_split3[14] == '음용온도'].shift(-6, axis=1) 
detail_split3[detail_split3[16] == '음용온도'] = detail_split3[detail_split3[16] == '음용온도'].shift(-8, axis=1)
detail_split3[detail_split3[18] == '음용온도'] = detail_split3[detail_split3[18] == '음용온도'].shift(-10, axis=1)


In [107]:
detail_split3

,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,알코올,13~14 %,음용온도,16~18 ℃,추천음식,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",수입사,None,None,None,None,None,None,None
1,None,None,음용온도,10~12 ℃,추천음식,"샐러드, 해산물, 치즈 등과 잘 어울린다.",기타정보,None,-international wine challenge 2013 bronze medal,-james halliday 94 points,None,None,None,None
2,None,None,음용온도,14~16 ℃,추천음식,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",None,None,None,None,None,None,None,None
3,알코올,20~21 %,음용온도,14~16 ℃,추천음식,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",수입사,None,None,None,None,None,None,None
4,알코올,14~15 %,음용온도,16~18 ℃,추천음식,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다.",수입사,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,알코올,14~15 %,음용온도,16~18 ℃,추천음식,"덩어리 째 오븐에 구운 육류와 햄, 스테이크, 라자냐, 치즈 등과 잘 어울린다.",기타정보,None,수입사,None,None,None,None,None
395,알코올,13~14 %,음용온도,8~10 ℃,추천음식,"연어 타르타르 스테이크와 같은 해산물 요리, 샐러드와 같은 야채, 생선 그릴 요리와...",기타정보,None,*Berlin Wine Trophy: Gold,수입사,None,None,None,None
396,알코올,11~12 %,음용온도,6~8 ℃,추천음식,"디저트, 조개, 생선 요리 또는 반주와 잘 어울린다.",기타정보,None,수입사,None,None,None,None,None
397,알코올,13~14 %,음용온도,8~10 ℃,추천음식,"샤프란을 곁들인 생선 스튜 요리 등 생선, 갑각류, 조개 요리, 크림 소스를 곁들인...",기타정보,None,None,None,None,None,None,None


In [109]:
df_wine["도수"] = detail_split3.get(7)
df_wine["음용온도"] = detail_split3.get(9)
df_wine["추천음식"] = detail_split3.get(11)

In [111]:
# 상세정보 열 삭제 
df_wine=df_wine.drop(['상세정보'],axis=1)

In [112]:
df_wine

,주종,중분류,상품명,생산국가,생산자,주요품종,도수,음용온도,추천음식
0,wine,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",13~14 %,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다."
1,wine,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,None,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다."
2,wine,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),None,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다."
3,wine,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",20~21 %,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다."
4,wine,레드,"기욤 르탕, 루비 루즈",프랑스,기욤 르탕 Guillaume Letang,그르나슈 (Grenache),14~15 %,16~18 ℃,"소고기, 스테이크, 불고기, 양꼬치, 족발, 한국음식, 피자, 햄버거 등과 잘 어울린다."
...,...,...,...,...,...,...,...,...,...
394,wine,레드,"매티스, 그르나슈",미국,매티스 와이너리 Mathis Winery,"그르나슈 (Grenache) 82%, 까리냥 (Carignan/Carignane) ...",14~15 %,16~18 ℃,"덩어리 째 오븐에 구운 육류와 햄, 스테이크, 라자냐, 치즈 등과 잘 어울린다."
395,wine,화이트,"메종 카스텔, 소비뇽 블랑",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),13~14 %,8~10 ℃,"연어 타르타르 스테이크와 같은 해산물 요리, 샐러드와 같은 야채, 생선 그릴 요리와..."
396,wine,스파클링,"메종 카스텔, 퀴베 블랑쉬 브뤼",프랑스,메종 카스텔 Maison Castel,샤르도네 (Chardonnay),11~12 %,6~8 ℃,"디저트, 조개, 생선 요리 또는 반주와 잘 어울린다."
397,wine,화이트,"메종 카스텔, 푸이 퓌메",프랑스,메종 카스텔 Maison Castel,소비뇽 블랑 (Sauvignon Blanc),13~14 %,8~10 ℃,"샤프란을 곁들인 생선 스튜 요리 등 생선, 갑각류, 조개 요리, 크림 소스를 곁들인..."


In [113]:
df_wine.to_csv('./wine400_fin.csv', encoding = 'utf-8-sig')